In [ ]:
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install -U transformers accelerate -q

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

tokenizer_config.json:   0%|          | 0.00/3.28k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    torch_dtype="auto",
    trust_remote_code=True,
    device_map="cuda",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch

In [ ]:
chat = [
    {
        "role":"user",
        "content":"What is NBA? Who won the previous edition of NBA Championship"
    }
]
prompt = tokenizer.apply_chat_template(chat, tokenize = False, add_generation_prompt = True)
token_id = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")


In [ ]:
with torch.no_grad():
  output_ids = model.generate(
      token_id.to(model.device),
      do_sample = True,
      temperature = 0.7,
      max_new_tokens = 512
  )

In [ ]:
output = tokenizer.decode(output_ids.tolist()[0][token_id.size(1):], skip_special_tokens=True)
output

"The NBA, or National Basketball Association, is a professional basketball league in North America. It is composed of 30 teams (29 in the United States and 1 in Canada) and is widely regarded as one of the premier men's professional basketball leagues in the world.\n\nThe NBA Championship is the annual championship trophy awarded to the winning team of the NBA Finals, which is the series that culminates at the end of each season between the Eastern and Western Conference champions. \n\nAs for the previous edition of the NBA Championship, it was won by the Golden State Warriors in the year 2022, where they defeated the Boston Celtics in a 4-2 series in the NBA Finals."

In [ ]:
# Function to interact with the user
def chat_with_user():
    chat_history = []

    while True:
        user_input = input("You: ")

        if user_input.lower() == "stop":
            break

        chat_history.append({"role": "user", "content": user_input})

        prompt = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
        token_id = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

        with torch.no_grad():
            output_ids = model.generate(
                token_id.to(model.device),
                do_sample=True,
                temperature=0.7,
                max_new_tokens=512
            )

        response = tokenizer.decode(output_ids.tolist()[0][token_id.size(1):], skip_special_tokens=True)

        chat_history.append({"role": "assistant", "content": response})

        print("Assistant:", response)

    print("Final chat history:")
    for entry in chat_history:
        print(f"{entry['role']}: {entry['content']}")


chat_with_user()

You: Hii
Assistant: Hello! How can I assist you today?
You: Myself Mohammed Raza who are you
Assistant: I am an AI digital assistant developed by Microsoft. I'm here to help answer any questions you might have to the best of my ability.
You: That's very great. On which last data and knowledge are you trained on
Assistant: As an artificial intelligence, my training data includes a diverse range of sources up to September 2021. I don't have the ability to access or retrieve personal data unless it has been shared in the course of our conversation, which I am designed to forget in order to maintain user privacy.
You: What is my name
Assistant: You've identified yourself as Mohammed Raza in our conversation.
You: What is your views on feminism
Assistant: As an artificial intelligence, I don't have personal opinions or feelings. However, I can share that feminism is a socio-political movement advocating for equal rights and opportunities for all genders. It seeks to challenge and ultimately